In [1]:
import os
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain_utils
import encryption

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb
Importing Jupyter notebook from encryption.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
identifier = utils.load_yaml('resources/identifier.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO THE ETHEREUM GATEWAY

In [6]:
web3 = blockchain_utils.connect(settings)

### SERIALIZE THE MANAGER CONTRACT

In [7]:
device_manager = blockchain_utils.contract(latest['devicemanager'], web3, settings)

### HASH THE DEVICE IDENTIFIER

In [8]:
hashed_identifier = utils.hash_data(identifier)

### USE THE HASHED IDENTIFIER TO LOCATE THE DEVICES SMART CONTRACT

In [9]:
device_address = device_manager.read({
    'func': 'fetch_device',
    'params': hashed_identifier
})

### SERIALIZE THE DEVICE SPECIFIC CONTRACT

In [10]:
device_contract = blockchain_utils.contract({
    'address': device_address,
    'abi': latest['device']['abi']
}, web3, settings)

### FETCH & SERIALIZE RSA ENCRYPTION KEY

In [11]:
def fetch_rsa():
    
    # FETCH MOST RECENT KEY
    raw_rsa_pubkey = device_contract.read('encryption_key')
    
    # SERIALIZE & RETURN
    return encryption.serialize_key(raw_rsa_pubkey, 'public')

### EVENTS

In [12]:
new_key_event = device_contract.event('new_key')

### EVENT LOOP

In [40]:
str_data = [
    'IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJh',
    'IjkxNjI5MGUxMzEyZjMzNDY1MTU1OWU4YTEyOGVlOTBlMTQwNzI5ZTJhMDBkYzUy',
    'IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJh',
    'IjkxNjI5MGUxMzEyZjMzNDY1MTU1OWU4YTEyOGVlOTBlMTQwNzI5ZTJhMDBkYzUy',
    'IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJh',
    'IjkxNjI5MGUxMzEyZjMzNDY1MTU1OWU4YTEyOGVlOTBlMTQwNzI5ZTJhMDBkYzUy',
    'IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJh',
    'IjkxNjI5MGUxMzEyZjMzNDY1MTU1OWU4YTEyOGVlOTBlMTQwNzI5ZTJhMDBkYzUy'
]

In [41]:
int_data = [1615915771, 1615915772, 1615915771, 1615915772, 1615915771, 1615915772, 1615915771, 1615915772]

In [44]:
len('IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJh')

64

In [43]:
device_contract.write({
    'func': 'evaluate',
    'params': [('foo', 'bar'), str_data, int_data]
})

ValueError({'code': -32000,
            'message': 'invalid opcode: opcode 0xfe not defined'})

In [13]:
    # OPEN THE FILE
    file = open(settings['log_path'], 'r')

    # FIND THE TAILEND OF THE FILE
    file_stats = os.stat(settings['log_path'])
    file_size = file_stats[6]
    
    # GOTO THE LAST ROW
    file.seek(file_size)
    
    # DATA CONTAINERS
    hashes = []
    timestamps = []
    batch = []
    
    # FETCH & SERIALIZE RSA PUBKEY
    rsa_pubkey = fetch_rsa()

    # EVENT LOOP
    while(True):

        # FETCH & SERIALIZE RSA PUBKEY WHEN UPDATED
        for event in new_key_event.get_new_entries():
            rsa_pubkey = fetch_rsa()

        # READ THE LAST LINE
        where = file.tell()
        line = file.readline()

        # IF THE LINE ISNT EMPTY
        if line:

            # EXTRACT THE EVENT PARAMS
            module, code, message, timestamp = utils.parse_line(line)
            
            # HASH THE MODULE & MESSAGE
            hashed = utils.hash_data({
                'module': module,
                'message': message
            })
            
            # ENCODE THE HASH TO BASE64
            encoded = utils.encode(hashed)
            
            # APPEND TO HASH & TIMESTAMP CONTAINERS
            hashes.append(encoded)
            timestamps.append(timestamp)

            # PUSH FULL ENTRY TO BATCH CONTAINER
            batch.append({
                'hash': encoded,
                'timestamp': timestamp,
                'module': module,
                'code': code,
                'message': message,
            })

        # IF THE TAILEND OF THE FILE WAS REACHED AND CONTAINER IS NOT EMPTY
        if file.tell() == os.path.getsize(settings['log_path']) and len(hashes)>0 and len(batch)>0:
            
            # AES ENCRYPT THE BATCH
            sym_data, sym_params = encryption.aes_encrypt({
                'events': batch
            })
            
            print(hashes)
            print(timestamps)

            # RSA ENCRYPT AES PARAMS
            asym_params = encryption.rsa_encrypt(sym_params, rsa_pubkey)

            # SUBMIT BATCH DATA TO THE SMART CONTRACT
            device_contract.write({
                'func': 'evaluate',
                'params': [(sym_data, asym_params), hashes, timestamps]
            })

            # PRINT MSG
            print('UPLOADED BATCH ({})'.format(len(batch)))

            # RESET CONTAINERS
            hashes = []
            timestamps = []
            batch = []
            
        # SLEEP FOR A SECOND
        time.sleep(1)

['IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJhNWIyNmFiMDAyZDM0ZmNlNzci', 'IjkxNjI5MGUxMzEyZjMzNDY1MTU1OWU4YTEyOGVlOTBlMTQwNzI5ZTJhMDBkYzUyMDE2YzE0YjNmZTZiZTc1Zjci', 'ImViYmEwOTBiOGE5NTFkYWU5Mjc1N2FlMzYxYjA2YTE0MzBlYzY2M2Q4ZGI4ZTAwNzYyMDhkODExOGFlY2ViZjci', 'ImFlZTFiZjNlNGU2NjhmNzRkZTlhMmYxYzg3NTg1NTkxYjM5NTA0NjIwNWU3MzQ5NWRlMGIzOGM5YTk1NTdiNGUi', 'IjM5MWFhMGM5NDJlZjJjZDc0ZjNkOGE3OWNkZTk5MWFkODNjN2QwNzkxOWVhMTJhNWIyNmFiMDAyZDM0ZmNlNzci', 'Ijg3ZDMzZDIzNGRkNjQyNjc3ZTJiNTc3Y2QzNjU5OGI0ODQyZGZhM2EzZTNiMWM5M2JlYWFkNzRkNWIxZTk0MGYi', 'IjE4MWMyNjg2MGEyOWExMGFhYzFhNDIxYzQ4NDA0MzdiNTRjMjI5ZmZmNGQ4ZTFkOWM2MDZhMWY4YTNlNzhjZWMi', 'IjhmYTA5MTQ2NGE3MDFlZDkyZmY3OTU2YTIzNmJkZWExNmQ5ZmYyNTMyZWE1ZGFkNGI4MzFjYzk5MGM4MGVhNDIi']
[1615915771, 1615915772, 1615915772, 1615915773, 1615915773, 1615915773, 1615915773, 1615915774]


ValidationError: 
Could not identify the intended function with name `evaluate`, positional argument(s) of type `(<class 'tuple'>, <class 'list'>, <class 'list'>)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `evaluate`: ['evaluate(tuple,string[],uint256[])']
Function invocation failed due to no matching argument types.